# Plotting of Evaluation Results: Decision Times

Compare number of decisions and time for decisions between different algorithms.
Take decisions per flow, node, and in total into account.

In [6]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

result_dir = 'decision_results'
rep = '56'
config = 'hc'
network = 'dfn_58.graphml'
ingresses = ['0.1']
algos = ['gpasp', 'spr2', 'bjointsp']

In [8]:
df = None
for algo in algos:
    for ing in ingresses:
        df_tmp = pd.read_csv(f'{result_dir}/{rep}/{config}/{network}/{ing}/{algo}/agg_decisions.csv')
        if df is None:
            df = df_tmp.copy()
        else:
            df.append(df_tmp)
        
df.head()

,Unnamed: 0,flow,node,dec_count,dec_time_avg,dec_time_std
0,0,1,pop2,4,0.000499,0.000499
1,1,1,pop49,1,0.000997,0.000000
2,2,1,pop48,1,0.000998,0.000000
3,3,1,pop52,1,0.000998,0.000000
4,4,1,pop51,1,0.000997,0.000000
